<center> <img src = images\hh_label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [4]:
# импорт необходимых библиотек, требующихся для выполнения всех заданий
# выключение предупреждений (предупреждение о том, что pandas поддерживает только SQLAlchemy connectable (engine/connection) или URL-адрес строки базы данных или подключение sqlite3 DBAPI2)

import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [5]:
# параметры для подключения к базе данных 
# удалены при размещении в открытом доступе на github
DBNAME = 'жу-жу-жу'
USER = 'бе-бе-бе'
PASSWORD = 'кря-кря-кря'
HOST = 'буль-буль'
PORT = '... и колонка юмора'

In [6]:
# блок параметров подключения для использования в pd.read_sql_query
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [7]:
# текст запроса
query_3_1 = f'''SELECT 
                    COUNT(id) vacancies
                FROM 
                    vacancies
             '''

In [8]:
# результат запроса
quantity_vacancies = pd.read_sql_query(query_3_1, connection)
quantity_vacancies

,vacancies
0,49197


2. Запрос, который посчитает количество работодателей (таблица employers). 

In [9]:
# текст запроса
query_3_2 = f'''SELECT 
                    COUNT(id) employers
                FROM 
                    employers
             '''

In [10]:
# результат запроса
quantity_employers = pd.read_sql_query(query_3_2, connection)
quantity_employers

,employers
0,23501


3. Запрос для получения количества регионов (таблица areas).

In [11]:
# текст запроса
query_3_3 = f'''SELECT 
                    COUNT(id) areas
                FROM 
                    areas
                '''

In [12]:
# результат запроса
quantity_areas = pd.read_sql_query(query_3_3, connection)
quantity_areas

,areas
0,1362


4. Запрос для получения количества сфер деятельности в базе (таблица industries).

In [13]:
# текст запроса
query_3_4 = f'''SELECT 
                    COUNT(id) industries
                FROM 
                    industries
             '''

In [14]:
# результат запроса
quantity_industries = pd.read_sql_query(query_3_4, connection)
quantity_industries

,industries
0,294


***

### Дополнительный анализ

6. Запрос для определения ТОП-50 работодателей с максимальным количеством вакансий. Так сказать звезды ИТ-работодателей.

In [15]:
# текст запроса
query_3_6 = f'''SELECT 
                    e.name,
                    COUNT(v.id) vacancies -- подсчет вакансий по их id для каждого работодателя 
                FROM 
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- присоединяем для получения названий работодателя
                GROUP BY e.id -- группируем по id работодателя
                ORDER BY vacancies DESC -- сортировка по убыванию
                LIMIT 50 -- ну и вывод первых 50 работодателей с максимумом вакансий
             '''

In [16]:
# результат запроса
top_50_vacancies = pd.read_sql_query(query_3_6, connection)
top_50_vacancies

,name,vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


7. Запрос для определения количества работодателей с не менее, чем 10 вакансиями.

In [17]:
# текст запроса
query_3_7 = f'''WITH tab AS ( -- таблица с работодателями у каждого из которых не менее 10 вакансий
                    SELECT 
                        COUNT(*) -- подсчет вакансий для каждого работодателя
                    FROM 
                        vacancies v
                        LEFT JOIN employers e ON v.employer_id = e.id -- добавляем название работодателя                 
                    GROUP BY v.employer_id -- группируем по id работодателя
                    HAVING COUNT(v.id) >= 10 -- условие не менее 10 вакансий у работодателя  
                )
                
                SELECT
                    COUNT(*) -- подстчет таких работодателей
                FROM
                    tab
             '''

In [18]:
# результат запроса
from_10_vacancies = pd.read_sql_query(query_3_7, connection)
from_10_vacancies

,count
0,730


8 Запрос для определения ТОП-100 регионов по количеству вакансий.

In [19]:
# текст запроса
query_3_8 = f'''SELECT 
                    a.name,
                    COUNT(v.id) vacancies -- подсчет вакансий по их id для каждого региона 
                FROM 
                    vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id -- присоединяем для получения названий регионов
                GROUP BY a.id -- группируем по id региона
                ORDER BY vacancies DESC -- сортировка по убыванию количества вакансий 
                LIMIT 100 -- ограничиваем первыми 100 регионами
             '''

In [20]:
top_100_areas_vacancies = pd.read_sql_query(query_3_8, connection)
top_100_areas_vacancies

,name,vacancies
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
95,Волжский (Волгоградская область),57
96,Обнинск,57
97,Одинцово,56
98,Якутск,54


9 Запрос для определения ТОП-100 регионов по количеству работодателей.

In [21]:
# текст запроса
query_3_9 = f'''SELECT 
                    a.name,
                    COUNT(e.id) employers -- подсчет работодателей по их id для каждого региона 
                FROM 
                    employers e
                    LEFT JOIN areas a ON e.area = a.id -- присоединяем для получения названий регионов
                GROUP BY a.id -- группируем по id региона
                ORDER BY employers DESC -- сортировка по убыванию количества работодателей
                LIMIT 100 -- ограничиваем первыми 100 регионами
             '''

In [22]:
top_100_areas_vacancies = pd.read_sql_query(query_3_9, connection)
top_100_areas_vacancies

,name,employers
0,Москва,5864
1,Санкт-Петербург,2217
2,Минск,1115
3,Алматы,721
4,Екатеринбург,609
...,...,...
95,Зеленоград,27
96,Новороссийск,26
97,Якутск,26
98,Сыктывкар,26


10. Запрос по количеству вакансий компаний из иностранных государств. Решено посмотреть среди стран СНГ, кроме Молдовы - они нас не любят, политика ... . Ну и плюс несколько "недружественных" государств - США, Германия, Польша, почему-то Кипр ? Море и солнце ?

In [23]:
# текст запроса
query_3_10 = f'''SELECT 
                     a.name,
                     COUNT(v.id) vacancies -- подсчет вакансий по их id для каждого региона 
                 FROM 
                     vacancies v
                     LEFT JOIN areas a ON v.area_id = a.id -- присоединяем для получения названий регионов
                 WHERE
                     a.name IN ('Россия', 'Кипр', 'США', 'Казахстан', 'Беларусь', 'Армения', 'Азербайджан', 'Киргизия', 'Таджикистан', 'Узбекистан', 'Германия', 'Польша', 'Кыргызстан') -- выбираем конкретные регионы из перечня
                 GROUP BY a.id -- группируем по id региона
                 ORDER BY a.name -- сортировка по регионам по алфавиту
              '''              
            

In [24]:
strange_areas = pd.read_sql_query(query_3_10, connection)
strange_areas

,name,vacancies
0,Армения,223
1,Германия,19
2,Кипр,136
3,Польша,53
4,США,12
5,Таджикистан,4


11 А вот пришла мысль в голову - все ли работодатели имеют хотя бы одну вакансию? Нет ли "счастливчиков", у которых все сеть и никто не нужен ? Нужно проверить.

In [25]:
# текст запроса
query_3_11 = f'''SELECT 
                    (SELECT COUNT(id) FROM employers) employers_all, -- все работодатели
                    (SELECT COUNT(DISTINCT employer_id) FROM vacancies) employers_honest, -- все уникальные id работодателей в таблице вакансий, т.е. все работодатели с вакансиями
                    (SELECT COUNT(id) FROM employers) - (SELECT COUNT(DISTINCT employer_id) FROM vacancies) Employers_fake -- разница, т.е. работодатели без вакансий
             '''                                

In [26]:
employers_fake = pd.read_sql_query(query_3_11, connection)
employers_fake

,employers_all,employers_honest,employers_fake
0,23501,14906,8595


12 Еще интересный вопрос возник, а сколько регионов вообще "не имеют работодателей" ? Нужно выяснить.

In [27]:
# текст запроса
query_3_12 = f'''SELECT
                    COUNT(a.id) areas_strange -- подсчитаем количество таких записей (т.е. где у региона нет ни одного работодателя)
                FROM
                    employers e 
                    FULL JOIN areas a ON e.area = a.id -- полное соединение таблиц работодателей и регионов
                WHERE e.id IS NULL -- оставим только записи, где id работодателя отсутствуют
             '''

In [28]:
areas_strange = pd.read_sql_query(query_3_12, connection)
areas_strange

,areas_strange
0,708


13 Для статистики, минимальную и максимальную зарплаты нужно узнать. Ну ладно, ладно, из любопытства, только нос надо поберечь, чтобы не оторвали.

In [29]:
# текст запроса
query_3_13_1 = f'''SELECT
                    salary_from
                FROM
                    vacancies
                WHERE salary_from IS NOT NULL -- минимальная указана
                ORDER BY salary_from -- сортировка по возрастанию
             '''

In [30]:
min_salary = pd.read_sql_query(query_3_13_1, connection)
min_salary

,salary_from
0,30
1,30
2,30
3,35
4,42
...,...
21480,500000
21481,574000
21482,574000
21483,650000


In [31]:
# текст запроса
query_3_13_2 = f'''SELECT
                    salary_to
                FROM
                    vacancies
                WHERE salary_to IS NOT NULL -- максимальная существует
                ORDER BY salary_to -- сортировка по возрастанию
             '''

In [32]:
max_salary = pd.read_sql_query(query_3_13_2, connection)
max_salary

,salary_to
0,60
1,75
2,350
3,850
4,1500
...,...
12685,935176
12686,950000
12687,1000000
12688,1000000


Для красоты сделаем визуализацию распределения вакансий по размеру зарплаты отдельно для нижнего и верхнего края ее же.

In [33]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


fig = go.Figure()

# две гистограммы для нажней границы и для верхней границы зарплаты, синенькая и красненькая, соответственно
fig.add_trace(go.Histogram(x=min_salary['salary_from'], name='нижняя граница', marker_color='#007dff', nbinsx=200))
fig.add_trace(go.Histogram(x=max_salary['salary_to'], name='верхняя граница', marker_color='#e75300', nbinsx=200))

# что показывать при наведении на графики
fig.update_traces(hoverinfo="all", hovertemplate="Зарплата: %{x}<br>Вакансии: %{y}")

# параметры с названием заголовка и отступы, плюс месторасположение легенды, ну и высота фигуры
fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Распределение вакансий по уровню зарплаты (нижней и верхней границам)', 
                  title_x = 0.5, title_font_size=25,
                  legend_orientation="h", legend=dict(x=.2, xanchor="center"),
                  height=600)

# параметры осей, названия, шрифты, размеры, то же для шкалы 
fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='Зарплата',
                 tickfont=dict(family='Rockwell', size=14),
                 dtick=50000)
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='Вакансии',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

<center> <img src = images\plot_3_13.png alt="drawing" style="width:2000px;">

### Выводы по предварительному анализу данных

* среднее количество вакансий у работодателя с вакансиями: 3.3 (общее количество вакансий 49197 и количество работодателей с вакансиями 14906). А вообще среднее число вакансий на всех работодателей: 2.1.
* максимальное количество вакансий составляет 1933 у компании Яндекс. Да, это "звезда по имени солнце". "Голубая мечта" начинающих ИТ-шников. Почти 4 % всех вакансий !!!
* 25 компаний имеют количество вакансий от 100 и более, это всего 0.1 % от общего количества работодателей с вакансиями. Ну это так, для "интереса".
* на 17 месте по количеству вакансий находится ИП Поляков Денис Иванович, единственная компания в ТОП-50 не относящаяся к **крупным** финансовым, ИТ, промышленным торговым и т.п. компаниям. Эту "темную лошадь" исследуем в "Дополнительных исследованиях". Да, тавтология (именно через "в" - Яндекс говорит!).
* количество компаний, которые разместили не менее 10 резюме - составляет 730. Можно считать, что это компании, которым требуется "существенное количество" IT специалистов. При этом часть компаний может быть агентствами по подбору персонала, ну а может и не быть. Если изменить порог минимального количества вакансий, то у 159 компаний - не менее 30 вакансий, у 81 компании - не менее 50 вакансий.
* распределение вакансий по регионам вполне ожидаемо - прямая корреляция с количеством населения, административным статусом, количеством экономических субъектов (компаний, предприятий, научных учреждений и т.д.). В первую сотню по количеству вакансий входят административные центры субъектов Российской Федерации, крупнейшие города помимо них, а также столицы и крупные города стран СНГ. Ну так, деревня Гадюкино не заказывает ИТ-спецов, там картоху растить нужно.
* распределение работодателей по регионам вполне ожидаемо и практически совпадает (по нахождению в первой сотне, не обязательно по конкретному месту) с распределением вакансий по регионам - прямая корреляция с количеством населения, административным статусом, количеством экономических субъектов (компаний, предприятий, научных учреждений и т.д.). В первую сотню по количеству вакансий входят административные центры субъектов Российской Федерации, остальные крупнейшие города, а также столицы и крупные города стран СНГ. Гм, ожидаемо, дамы и господа, ожидаемо.
* внезапно выяснилось, что среди перечня регионов есть страны СНГ (за исключением Молдовы, вот всегда они "хитрые молдаване"), Россия (зачем ??), а также совсем чуждые: Германия, Кипр, США и Польша, например, простите - "недружественные". При этом вакансии указаны для: Армении   223, Германии   19, Кипра   136, Польши 53, США 12 и Таджикистана 4. Также можно отметить, что не указаны вакансии для Беларуси, Казахстана, России, Азербайджана, Киргизии, Узбекистана и Кыргызстана - как стран в целом. Хотя для их регионов (городов) вакансии присутствуют. Т.е. налицо определенная путаница с присутствием в справочнике регионов государств, наряду с регионами.
* ну и еще интересный факт, среди 1362 регионов 708 не имеют работодателей (согласно данной базе, разумеется). Это аж 52 %.
* нижняя граница указана для 21485 вакансий (43.7 % всех вакансий) и составляет от 30 до 1000000 руб. Значение в миллион - единственное, предпоследнее 650 тыс. и уже 5 с конца - 500 тыс. Далее за 200 значений до конца уже менее 300 тыс. Т.е. значения в несколько сотен тысяч - это лишь небольшая часть. Примерно такая же картина и для верхней границы. Она указана лишь для 12690 вакансий (25.8 % всех вакансий) и составляет от 60 до 1000000 рублей. Миллион в трех крайних значениях, далее к 20 с конца значению уже видим половину миллиона, к 200 значению с конца - 350 тыс. руб. Т.е. падает, но помедленнее, чем для нижней границы зарплаты. При этом очевидно, что в минимальных значениях пропущены нули или разряд "тысячи". Исходя из здравого смысла следует ориентироваться на минимальные значения от 15 - 20 тыс. рублей.

---

# Юнит 4. Детальный анализ вакансий

1. Запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area). Сортировка по количеству вакансий в порядке убывания.

In [34]:
# текст запроса
query_4_1 = f'''SELECT
                    a.name,
                    COUNT(v.id) cnt -- подстчет количества по id вакансий
                FROM
                    vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id -- получаем названия регионов для таблицы с вакансиями
                GROUP BY a.name -- группируем по регионам
                ORDER BY cnt DESC 
             '''

In [35]:
# результат запроса
area_vacancies = pd.read_sql_query(query_4_1, connection)
area_vacancies

,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой. Чуть ниже "расширенная" версия.

In [36]:
# текст запроса
query_4_2 = f'''SELECT
                    COUNT(id)
                FROM
                    vacancies 
                WHERE
                    salary_from IS NOT NULL OR salary_to IS NOT NULL -- заполнено хотя бы одно поле с зарплатой 
             '''

In [37]:
# результат запроса
salary_not_empty = pd.read_sql_query(query_4_2, connection)
salary_not_empty

,count
0,24073


3.Нахождение средних значений для нижней и верхней границы зарплатной вилки. Округление значения до целого.

In [38]:
# текст запроса
query_4_3 = f'''SELECT
                    ROUND(AVG(salary_from))::int avg_salary_from, -- для крастоы приводим к типу int
                    ROUND(AVG(salary_to))::int avg_salary_to -- аналогично
                FROM
                    vacancies
             '''

In [39]:
# результат запроса
avg_salary = pd.read_sql_query(query_4_3, connection)
avg_salary

,avg_salary_from,avg_salary_to
0,71065,110537


4. Запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат сортирован по убыванию количества. Для удобства анализа вводится дополнительное поле (столбец таблицы) с долей каждого сочетания (графика и трудоустройства) в процентах от общего количества вакансий. 


In [40]:
# текст запроса
query_4_4 = f'''WITH sched_employment AS ( -- это основная таблица
                    SELECT
                        schedule, 
                        employment,                     
                        COUNT(id) vacancies
                    FROM
                        vacancies 
                    GROUP BY schedule, employment -- группировка по графику, а внутри по типу занятости
                )
                SELECT
                    schedule, 
                    employment,                     
                    vacancies,
                    ROUND(vacancies/(SELECT SUM(vacancies) FROM sched_employment)*100, 5) percent -- вычисляем процент от общего количества вакансий, сильно округлить нельзя - петеряются числа около нуля
                FROM
                    sched_employment
                ORDER BY vacancies DESC -- сортируем, как указано
             '''

In [41]:
# результат запроса
sched_employment = pd.read_sql_query(query_4_4, connection)
sched_employment

,schedule,employment,vacancies,percent
0,Полный день,Полная занятость,35367,71.88853
1,Удаленная работа,Полная занятость,7802,15.85869
2,Гибкий график,Полная занятость,1593,3.23800
3,Удаленная работа,Частичная занятость,1312,2.66683
4,Сменный график,Полная занятость,940,1.91069
5,Полный день,Стажировка,569,1.15657
6,Вахтовый метод,Полная занятость,367,0.74598
7,Полный день,Частичная занятость,347,0.70533
8,Гибкий график,Частичная занятость,312,0.63419
9,Полный день,Проектная работа,141,0.28660


5. Запрос, выводящий значения поля требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. Аналогично для удобства анализа вводится дополнительное поле (столбец таблицы) с долей каждого сочетания (графика и трудоустройства)значения в процентах от общего количества вакансий (на самом деле чтобы не открывать калькулятор и вручную не считать все эти проценты (хотя не факт, что затраты времени себя окупают, но ... "красиво")).

In [42]:
# текст запроса
query_4_5 = f'''WITH exp_how_many AS ( -- это основная таблица
                    SELECT
                        experience,
                        COUNT(id) quantity
                    FROM
                        vacancies
                    GROUP BY experience -- группировка по опыту
                )
                SELECT
                    experience,
                    quantity,
                    ROUND(quantity/(SELECT SUM(quantity) FROM exp_how_many)*100, 1) percent -- вычисляем процент от общего количества вакансий, ну и слегка округляем
                FROM
                    exp_how_many
                ORDER BY quantity -- сортируем, как заказано
            '''

In [43]:
# результат запроса
experience_vacancies = pd.read_sql_query(query_4_5, connection)
experience_vacancies

,experience,quantity,percent
0,Более 6 лет,1337,2.7
1,Нет опыта,7197,14.6
2,От 3 до 6 лет,14511,29.5
3,От 1 года до 3 лет,26152,53.2


***

### Дополнительный анализ

6. Запрос у какого количества вакансий заполнены  или не заполнены одно или оба поля с зарплатой. Конечно же с процентами, мы же "за красоту".

In [44]:
# текст запроса
query_4_6 = f'''WITH salary_from_to AS ( -- делаем таблицу с 4 строками методом объединения 4 однострочных таблиц
                    SELECT
                        'Минимум и Максимум' salary,
                        COUNT(id) vacancies
                    FROM
                        vacancies 
                    WHERE
                        salary_from IS NOT NULL AND salary_to IS NOT NULL -- заполнены оба поля с зарплатой
                    UNION -- объединяем (и далее также)
                    SELECT
                        'только Минимум',
                        COUNT(id)
                    FROM
                        vacancies 
                    WHERE
                        salary_from IS NOT NULL AND salary_to IS NULL -- заполнено только поле с минимумом зарплаты
                    UNION
                    SELECT
                        'только Максимум',
                        COUNT(id)
                    FROM
                        vacancies 
                    WHERE
                        salary_from IS NULL AND salary_to IS NOT NULL -- заполнено только поле с максимумом зарплаты
                    UNION
                    SELECT
                        'Отсутствует',
                        COUNT(id)
                    FROM
                        vacancies 
                    WHERE
                        salary_from IS NULL AND salary_to IS NULL -- не заполнены оба поля с зарплатой 
                )
                SELECT -- итоговая таблица с добавленным столбцом "процент от общего количества вакансий"
                    salary,
                    vacancies,
                    ROUND(vacancies/(SELECT SUM(vacancies) FROM salary_from_to)*100)::int percent -- вычисление процента вакансий с (не)указанием минимума\максимума зарплаты с округлением и приведением к целому
                FROM
                    salary_from_to
                ORDER BY vacancies DESC -- сортировка по убыванию количества вакансий
             '''

In [45]:
# результат запроса
salary_full_empty = pd.read_sql_query(query_4_6, connection)
salary_full_empty

,salary,vacancies,percent
0,Отсутствует,25124,51
1,только Минимум,11383,23
2,Минимум и Максимум,10102,21
3,только Максимум,2588,5


### Выводы по детальному анализу вакансий

* распределение вакансий по регионам, как уже ранее показано, прямо коррелирует с количеством населения, административным статусом, количеством экономических субъектов (компаний, предприятий, научных учреждений и т.д.) в данном регионе. В первую сотню по количеству вакансий входят административные центры субъектов Российской Федерации, крупнейшие города кроме них, а также столицы и крупные города стран СНГ. Так на долю Москвы приходится почти 11 % от общего количества вакансий, доля Санкт-Петербурга почти 6 %. На третьем месте Минск с более, чем 4 %. Вот так и "Опустела без тебя земля ..." - все едут "столицу покорять" и крупные города, вакансии то там.
* более половины вакансий (51 % - все-таки более) не имеет каких-либо указаний на предлагаемую заработную плату, не минимума ни максимума (то ли работодатель считает, что эта работа вообще ничего не стоит, то ли наоборот, готов отдать бесконечные богатства за ее выполнение). В 21 % имеется полная информация о предложении работодателя по зарплате. В отношении только минимума определились еще в 23 % вакансий. В 5 % вакансий работодатели точно знают больше какой суммы не заплатят ни при каких условиях, а вот меньше - пожалуйста. Скорее всего это говорит о том, что работодатели ставят себя выше соискателей, они как-бы снисходят к пожеланиям и мечтам о получении работы: "Ну сколько там вы хотите, на что вы рассчитываете, как вы себя оцениваете ...". Все-таки вакансии с описанием требований, обязанностей, графика и иных деталей, но без указания предлагаемой заработной платы сложно воспринимать как полноценные адекватные предложения. Зарплата - принципиально существенное условие.
* средние значения для нижней и верхней границы зарплатной вилки составляют 71 и 110 тыс. рублей, соответственно (правда мы-то помним о том, что только половина работодателей вообще хоть что-то указала о зарплате). Отрадно, что в этом вопросе, в среднем, разумеется, присутствует определенность. Половина работодателей (как указано выше) напряглась и выдала тайну своей зарплатной щедрости/жадности, а 21 % так и вообще позволяет соискателям планировать свое будущее явно сообщая сколько готов платить "от сих до сих". Это действительно ценно. А вот абсолютные значения не радуют. Опять мы отстаем от т.н. развитого мира. Дополнительно можно отметить, что уровень средних зарплат, полученный из этой базы данных не совсем соответствует реальным зарплатам в ИТ сфере в России на 2022 год. [По этим данным](https://habr.com/ru/article/679698/), ну как пример, реальная медианная зарплата за первое полугодие 2022 года составила 140 тыс. рублей. Возможны две причины - зарплаты в вакансиях работодателей в-принципе ниже реальных рыночных, а также разные совокупности искомых сотрудников\работников и предоставивших информацию о своей заработной плате (а выборка достаточно большая - 12 тыс. работающих в ИТ сфере человек). Если сравнивать с дальним зарубежьем, то наши работодатели о скромны в зарплатных предложениях. [Например](https://visasam.ru/emigration/rabota/zarplaty-programmistov-v-mire.html), от 180 тыс. руб. в Германии до 450 тыс. в США, в среднем (конечно влияет курс рубля и иные и технические и политические причины).
* распределение вакансий для каждого сочетания типа рабочего графика и типа трудоустройства показывает явное предпочтение работодателя к тотальному физическому контролю над своим наемным персоналом. Более 80 % вакансий предполагают личное присутствие на рабочем месте (полный день, сменный и гибкий график), на долю удаленной работы приходится лишь около 19 %. Вахтовый метод где-то 0.8 %, да и вообще некая "странность" для ИТ сферы. Не совсем понятно, что имеется ввиду, нужно смотреть конкретные вакансии. Относительно формата занятости опять ожидаемо - более 93 % вакансий предполагают полную занятость. Если работодатели вас покупают, то желают иметь целиком, ну в смысле все ваше рабочее (да и нерабочее подчас) время. Частичная занятость и стажировка - всего чуть больше 4 и полутра процентов, соответственно. Ну а проектная работа, менее 1 %, это, чувствуется, совсем экзотика для нашего трудового рынка.
* требуемый опыт работы указан в подавляющем количестве вакансий - более 85 %. Ну что же, один из стандартных способов перестраховки при найме. Опыт как-бы свидетельствует о качестве нанимаемого лица, как-бы гарантирует, что руководство, хозяин, ну и HR-специалист не "сядут в калошу". Опыт как-бы говорит: "Ну, если уж, его там столько держали, ..., значит он может, значит и мы можем". Вот только действительно "как-бы", не 100% гарантия. Да и замечательным и великолепным новичкам без опыта, но "с холодной головой, горячим сердцем и чистыми руками" как быть-то? И вот тут-то, вот тут-то ... Приходят на помощь они - 14,6 % вакансий, где опыт не указан, ну совсем. Правда подлые мысли гложут: ~~а если бы он вез патроны ?~~ а если это либо совсем низкооплачиваемая непрестижная работа ? или наоборот, очень даже высокооплачиваемая, но с такими требованиями по навыкам, образованию, участию в космических проектах, фотосессиям с президентами, не менее, чем четырьмя иностранными языками в совершенстве, признанием святости Ватиканом ... ?? И так бывает, да (как раньше требовали два языка для вакансии уборщицы в банке). Правда работодатели все-таки дают послабление: опыт требуют, но какой именно? Половину устраивает опыт от 1 до 3 лет. А почти 30 % по строже, но все-таки говорят: "от 3 до 6" (прямо как у следователя на допросе). И только о 2.7 % работодателей можно сказать "кремень", только больше 6 лет опыта. Но все равно, смотрите какой прогресс и завоеванные трудовые права - лет ...цать назад бывали вакансии с требованием "до 25 лет, образование высшее, опыт от 10 лет" - это когда нужно было ВУЗ заканчивать? в 15? Или детский рабский труд? Нет, товарищи, все-таки "лучше меньше, да лучше".


# Юнит 5. Анализ работодателей

1. Запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий. Ну вот захотели так.

In [46]:
# текст запроса
query_5_1 = f'''WITH employers AS ( -- тут создадим таблицу с количеством вакансий у каждого работодателя, с сортировкой по убыванию
                    SELECT
                        e.name,
                        COUNT(v.id) quantity
                    FROM
                        vacancies v
                        LEFT JOIN employers e ON v.employer_id = e.id -- как обычно получаем названия работодателей
                    GROUP BY e.id
                    ORDER BY quantity DESC
                )
                (SELECT -- тут возьмем 1 строку из созданной таблицы
                     name,
                     quantity,
                     1 place
                 FROM
                     employers
                 LIMIT 1)
                UNION -- тут объединим две нужные строки в новую табличку
                (SELECT -- тут возьмем 5 строку из созданной таблицы
                     name,
                     quantity,
                     5
                 FROM
                     employers
                 OFFSET 4
                 LIMIT 1)
                 ORDER BY place -- отсортируем по месту, для красоты опять же
            '''

In [47]:
# результат запроса
employer_vacancies = pd.read_sql_query(query_5_1, connection)
employer_vacancies

,name,quantity,place
0,Яндекс,1933,1
1,Газпром нефть,331,5


2. Запрос, который для каждого региона выведет количество работодателей и вакансий в нём. Среди регионов, в которых нет вакансий, найдем тот, в котором наибольшее количество работодателей. Здесь удалим регионы для которых нет ни одного работодателя (и ни одной вакансии, соответственно).


In [48]:
# текст запроса
query_5_2 = f'''SELECT
                    a.name,
                    vacan.vacan_quantity,
                    employ.employ_quantity
                FROM areas a -- присоединяем слева к таблице с регионами таблицы vacan и employ по ID регионов, получим для каждого региона количество вакансий и работодателей
                     LEFT JOIN (SELECT 
                                    a.id id,
                                    COUNT(v.id) vacan_quantity
                                FROM
                                    vacancies v 
                                    LEFT JOIN areas a ON v.area_id = a.id
                                GROUP BY a.id -- таблица vacan с названиями регионов и количеством вакансий по каждому
                                ) vacan ON a.id = vacan.id
                     LEFT JOIN (SELECT
                                    a.id id,
                                    COUNT(e.id) employ_quantity
                                FROM
                                    employers e 
                                    LEFT JOIN areas a ON e.area = a.id
                                GROUP BY a.id -- таблица employ с названиями регионов и количеством работодателей в них
                                ) employ ON a.id = employ.id
                WHERE employ_quantity IS NOT NULL -- оставим только те регионы, в которых есть хотя бы один работодатель
                ORDER BY vacan_quantity DESC, employ_quantity DESC -- сортируем по количеству вакансий по убыванию, при этом все отсутствующие будут сверху, а после этой сортировки сортируем еще по количеству работодателей по убыванию и тогда в первой строке получим требуемый регион
             '''

In [49]:
# результат запроса
area_vacan_employ = pd.read_sql_query(query_5_2, connection)
area_vacan_employ

,name,vacan_quantity,employ_quantity
0,Россия,NaN,410
1,Казахстан,NaN,207
2,Московская область,NaN,75
3,Краснодарский край,NaN,19
4,Ростовская область,NaN,18
...,...,...,...
649,Электроугли,1.0,1
650,Белореченск,1.0,1
651,Красный Сулин,1.0,1
652,Заволжье,1.0,1


3. Для каждого работодателя посчитаем количество регионов, в которых он публикует свои вакансии. Отсортируем результат по убыванию количества.

In [50]:
# текст запроса
query_5_3 = f'''SELECT
                    e.name,
                    COUNT(DISTINCT v.area_id) quantity -- количество регионов для каждого работодателя
                FROM
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- просто добавляем слева для названия работодателя
                GROUP BY e.id -- группируем по id работодателя
                ORDER BY quantity DESC
            '''

In [51]:
# результат запроса
employ_vacan_area = pd.read_sql_query(query_5_3, connection)
employ_vacan_area


,name,quantity
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишем запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [52]:
# текст запроса
query_5_4 = f'''SELECT
                    COUNT(*)
                FROM
                    employers e
                    FULL JOIN employers_industries e_i ON e.id = e_i.employer_id -- полное соединение, получаем записи (строки) для каждого работодателя со всеми id сферы деятельности (построчно) или одну запись с "пустышкой" на месте id сферы деятельности (если для id работодателя нет ни одной записи в таблице employers_industries)
                WHERE 
                    e_i.industry_id IS NULL -- оставляем только записи с пустыми полями id сферы деятельности (а это как раз работодатели с отсутсвием сфер деятельности)
            '''

In [53]:
# результат запроса
employ_industies_empty = pd.read_sql_query(query_5_4, connection)
employ_industies_empty

,count
0,8419


5. Напишем запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. Так захотелось.

In [54]:
# текст запроса
query_5_5 = f'''SELECT
                    e.name
                FROM
                    employers_industries e_i
                    LEFT JOIN employers e ON e_i.employer_id = e.id -- просто добавляем названия и id работодателей
                GROUP BY e.id -- а потом группируем по id работодателя
                HAVING COUNT(e_i.industry_id) = 4 -- оставляем только тех работодателей, у которых 4 записи, т.е. 4 сферы деятельности
                ORDER BY name -- сортировка по алфавиту
                OFFSET 2 -- отступ на 2 строки
                LIMIT 1 -- вывод первой после отступа
            '''

In [55]:
# результат запроса
third_employ_4_industies = pd.read_sql_query(query_5_5, connection)
third_employ_4_industies

,name
0,2ГИС


6. С помощью запроса выясним, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения. Причем именно это и больше ничего. 


In [56]:
# текст запроса
query_5_6 = f'''SELECT
                    COUNT(DISTINCT employer_id) quantity -- подсчет количества уникальных, т.е. неповторяющихся id работодателей
                FROM
                    employers_industries e_i
                    LEFT JOIN industries i ON e_i.industry_id = i.id -- добавляем сами названия сфер деятельности
                WHERE i.name ~ 'Разработка программного обеспечения' -- только те работодатели, у кого указано требуемое (точно эта фраза)
             '''

In [57]:
# результат запроса
employ_certain_industry = pd.read_sql_query(query_5_6, connection)
employ_certain_industry

,quantity
0,3553


7. Для компании «Яндекс» выведем список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавим строку Total с общим количеством вакансий компании. Результат отсортируем по возрастанию количества. 
Список городов-милионников взьмем [отсюда](https://ru.wikipedia.org/wiki/Города-миллионеры_России).

In [58]:
# код для получения списка городов-милионников

# импорт необходимых библиотек
import requests
from bs4 import BeautifulSoup


# извлечение данных со страницы по указанному адресу
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url).text

# создание объекта типа BeautifulSoup с использованием парсера lxml
# потом методом select получаем содержимое нужной таблицы с городами-миллионниками - это список
# дальше для 1 элемента списка точно также получаем список содержимого для тегов a с атрибутами href и title
soup = BeautifulSoup(response, 'lxml')
tab_content = soup.select('#mw-content-text > div.mw-parser-output > table.standard.sortable > tbody')
cities = tab_content[0].select('a[href][title]')

# в результате получено искомое, но есть два лишних элемента - две ссылки в названиях столбцов таблицы, 
# удовлетворяющих условию, нужно их удалить
# подготовим новый пустой список
city_list = []

# в цикле проверим все элементы имеющегося списка на условия - 
# не больше двух слов (вроде нет названий городов из трех и более слов) и первая буква заглавная
for i in cities:
    elem = i.text
    if (len(elem.split()) <= 2) & (elem[0].isupper()):
        city_list.append(elem)

# преобразуем список в кортеж и выведем
city_list = tuple(city_list)

print(city_list)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [59]:
# текст запроса
query_5_7 = f'''WITH tab AS (
                SELECT
                    a.name,
                    COUNT(*)::int cnt
                FROM
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id
                    LEFT JOIN areas a ON v.area_id = a.id -- в таблицу вакансий добавляем названия работодателей и регионов, методом присоединения слева
                WHERE 
                    e.name LIKE 'Яндекс' -- условие: только Яндекс
                    AND a.name IN {city_list} -- условие: только из списка миллионников точнее кортежа
                GROUP BY a.id
                ) -- cоздаем таблицу tab с регионами из заданного списка (города-миллионники) и количеством вакансий для каждого, исключительно для Яндекса
                
                SELECT 
                    *
                FROM
                    tab
                UNION -- добавляем к имеющейся таблице tab общее количество вакансий по всем городам-миллионникам из нее же
                SELECT
                    'Total',
                    SUM(cnt) -- общая сумма вакансий
                FROM
                    tab
                ORDER BY cnt -- ну и сортируем по количеству вакансий по возрастанию
             '''

In [60]:
# результат запроса
yandex_million_areas = pd.read_sql_query(query_5_7, connection)
yandex_million_areas

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


### Дополнительный анализ

8 В каких регионах Яндекс публикует свои вакансии ? Сейчас разведка доложит точно ...

In [61]:
query_5_8 = f'''SELECT
                    a.name, -- выводим названия регионов
                    COUNT(v.id) quantity -- считаем количество вакансий по их id для каждого региона
                FROM
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- просто в вакансии добавляем сами названия работодателей
                    LEFT JOIN areas a ON v.area_id = a.id -- также добавляем сами названия регионов
                WHERE e.name = 'Яндекс' -- оставляем только Яндекс
                GROUP BY a.id -- группировка по id региона
                ORDER BY quantity DESC -- сортируем по количеству вакансий по убыванию, спорю на первом месте будет Москва ... даааааа                          
            '''

In [62]:
yandex_areas = pd.read_sql_query(query_5_8, connection)
yandex_areas

,name,quantity
0,Москва,54
1,Санкт-Петербург,42
2,Екатеринбург,39
3,Нижний Новгород,36
4,Новосибирск,35
...,...,...
176,Кызылорда,1
177,Томилино,1
178,Костанай,1
179,Актобе,1


### Выводы по анализу работодателей

* работодатели на первом и пятом месте по количеству вакансий: Яндекс и Газпромнефть. Ранее мы уже изучали ранжирование работодателей по количеству вакансий, это просто частный случай. Но еще раз похвалим "системообразующий" Яндекс - крепко держит ИТ сферу в России, СНГ и русскоязычном интернете за ... "шиворот" ??
* количество работодателей и вакансий для каждого региона мы тоже уже видели ранее, по отдельности. Оказалось, что 186 регионов без вакансий, но об этом тоже говорили. Здесь страны, и другие составные субъекты, отдельные части которых присутствуют в перечне регионов. Плюс маленькие регионы, без текущей потребности в ИТ специалистах.
* если посмотреть количество регионов для каждого работодателя, в которых он публикует свои вакансии, то мы видим корреляцию с количеством вакансий у работодателей. У кого много вакансий - у того много и регионов, в которых он публикует свои вакансии. У кого одна вакансия - тот опубликовал ее в одном регионе, удивительно, правда? Ну, Data Science еще и не такое может. Что отметим? Вездесущий Яндекс ожидаемо на первом месте c результатом 181 регион. Интересно в каких регионах он публикует вакансии и вообще, что это "регион публикации" - местонахождение рабочего места? местонахождение соискателя? место жительства для удаленной работы? Наверно, в первую очередь, это место работы для типа рабочего графика (полный день, сменный и гибкий график, вахтовый метод). А вот для удаленной работы - не имеет смысла. Теперь про Яндекс, дополнительно исследуем. Где размещены вакансии: Москва, СПб, все административные центры субъектов РФ, самые крупные города, плюс тоже самое по Казахстану и Беларуси. Стала интересна разбивка вакансий по регионам, а что там. Ничего интересного, наверно можно называться Нострадамусом. Полная корреляция с величиной и статусом региона, первые пять: Москва 54, Санкт-Петербург 42, Екатеринбург 39, Нижний Новгород 36, Новосибирск 35. Правда и Кызыл-орде досталась вакансия.
* продолжаем удивляться лени работодателей, почти 8.5 тыс. из них не указали сферы деятельности в которых они предлагают работу, а это без малого 36 % всех "акул капитализма". Конечно название вакансии и описание ключевых навыков может многое сказать, но что же так лениться?
* ну что, давайте поздравим компанию 2ГИС, она указала 4 сферы деятельности.
* у какого количества работодателей в качестве сферы деятельности указана "Разработка программного обеспечения", причем только эта фраза и больше ничего? Наука умеет много гитик. У 3553 работодателей указана такая сфера деятельности. Вы им верите ?? 
* посмотрим количество вакансий Яндекса по городам-миллионникам, которых на момент было 16. Опять без неожиданностей, Москва первая, СПб второй и т.д. Всего 485 вакансий, что составляет четверть от всех вакансий Яндекса. Ну молодец Яндекс, в-принципе не забывает и другие регионы, поскромнее.
* и опять подивимся некоему ИП Поляков Денис Иванович, который оказался на 4 месте среди работодателей по количеству регионов опубликованных вакансий, коих целых 88. Ничего, мы уже скоро о нем все узнаем.

# Юнит 6. Предметный анализ

1. Выясним сколько вакансий имеет отношение к данным? Зачем - так надо. Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн', причем в любом регистре. 


In [63]:
# текст запроса
query_6_1 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies v
                WHERE 
                    lower(v.name) ~* 'data' OR lower(v.name) ~* 'данн' -- приводим значение поля к нижнему регистру и проверяем наличие двух заданных эелементов в нем
             '''

In [64]:
# результат запроса
data_vacancies = pd.read_sql_query(query_6_1, connection)
data_vacancies

,count
0,1771


2. Подсчитаем сколько есть подходящих вакансий для начинающего дата-сайентиста. Порой так хочется всплакнуть, когда вакансий очень мало ...

Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

***В следующих заданиях будем использовать это условие.***

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' ***или***
* требуемый опыт — Нет опыта ***или***
* тип трудоустройства — ***Стажировка***
 

In [65]:
# текст запроса
query_6_2 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (lower(name) ~* 'junior' OR
                    experience ='Нет опыта' OR
                    employment = 'Стажировка') -- применение условия для уровня 'junior': наличие элемента 'junior' в поле name (причем в любом регистре, для этого приводим название вакансии к нижнему регистру), для опыта и занятости соответствие заданным значениям
             '''

In [66]:
# результат запроса
junior_vacancies = pd.read_sql_query(query_6_2, connection)
junior_vacancies

,count
0,51


3.Теперь узнаем сколько есть вакансий для DS (дата-сайентистов), в которых в качестве ключевого навыка указан SQL или postgres. Мы такие любознательные, загадочные ...

***Критерии для отнесения вакансии к DS указаны ранее.***

In [67]:
# текст запроса
query_6_3 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (key_skills ~* 'SQL' OR
                    key_skills ~* 'postgres') -- соответствие условию наличия двух ключевых навыков, причем в любом месте значения поля (с учетом регистра)
             '''

In [68]:
# результат запроса
sql_vacancies = pd.read_sql_query(query_6_3, connection)
sql_vacancies

,count
0,201


4. Проверим насколько популярен Python в требованиях работодателей к DS. Python... как много в этом звуке, Для сердца русского слилось! Как много в нем отозвалось! (Примечание: в Python попробуйте ударение на последнем слоге). Для этого узнаем количество вакансий, в которых в качестве ключевого навыка указан Python. 

***Критерии для отнесения вакансии к DS указаны ранее.***

In [69]:
# текст запроса
query_6_4 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (key_skills ~ 'Python')-- соответствие условию наличия ключевого навыка, причем в любом месте значения поля (с учетом регистра)
             '''

In [70]:
# результат запроса
python_vacancies = pd.read_sql_query(query_6_4, connection)
python_vacancies

,count
0,351


5. Теперь выясним сколько ключевых навыков в среднем указывают в вакансиях для DS? Ответ округлим до двух знаков после точки-разделителя. "Холмс, неужели это возможно ?", "Дорогой мой Ватсон ! Мой дедуктивный метод, да после тарелки овсянки ...".

In [71]:
# текст запроса
query_6_5 = f'''SELECT
                    ROUND(AVG(length(key_skills) - length(replace(key_skills, '\t', '')) + 1), 2) skills -- считаем количество символов в строке и вычитаем количество символов в строке, где удалили символы разделители '\', т.о. вычисляем количество символов разделитетелей, да + 1 (так как разделитетелей на 1 меньше чем разделяемых элементов), ну и считаем среднее с округлением до 2 знаков
                FROM
                    vacancies
                WHERE
                    name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*' -- уже давно известное условие соответствия вакансии дата-сайентиста
             '''

In [72]:
# результат запроса
avg_skills = pd.read_sql_query(query_6_5, connection)
avg_skills

,skills
0,6.41


6. Запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникального значения). Ооо, это то, за чем охотятся ЦРУ с Моссадом.

Устанавливаем следующие условия:
1. Рассматриваем только те вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой (минимальная и максимальная). А вы хотели, что бы мы посчитали среднее для пустоты ? Да ? 
2. Если заполнены оба поля с зарплатой, то берем среднеарифметическое значений минимальной и максимальной зарплаты. Если заполнено только одно из полей, берем его значение.
3. Для исключения влияния null будем использовать функцию coalesce, которая заменит null на передаваемое в нее значение.

Выясним, на какую зарплату, в среднем, может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлим до целого числа. 

In [73]:
# текст запроса
query_6_6 = f'''SELECT
                    experience,
                    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2))::int avg_salary -- слагаемые дадут нам или две минимальных или две максимальных, ну или минимальную и максимальную зарплаты, да на 2 поделим, вот и будет или средняя или просто минимум или максимум
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- опять же условие для вакансий именно дата-сайентистов
                    AND
                    (salary_from IS NOT NULL OR salary_to IS NOT NULL) -- условие наличия данных хотя бы для минимальной или максимальнной зарплаты
                GROUP BY experience -- сгруппируем по опыту, а иначе как среднее посчитать-то ?
                ORDER BY avg_salary DESC -- посортируем слегка, по убыванию (по ним, по ним, по денюжкам)
             '''

In [74]:
# результат запроса
avg_salary = pd.read_sql_query(query_6_6, connection)
avg_salary

,experience,avg_salary
0,От 3 до 6 лет,243115
1,От 1 года до 3 лет,139675
2,Нет опыта,74643


***

### Дополнительный анализ

7. А вот интересно, как там с когнитивными функциями у сотрудников и владельцев работодателей ? Есть ли что-то между количеством ключевых навыков и зарплатой ?? Ну, типа, как-бы связь, порочная или незапятнанная. Зарплату опять придется смотреть только существующую.

In [75]:
# текст запроса
query_6_7 = f'''SELECT
                    (length(key_skills) - length(replace(key_skills, '\t', '')) + 1)::int skills, -- опять вычисляем количество символов разделитетелей, да + 1 (так как разделитетелей на 1 меньше чем разделяемых элементов)
                    COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from) / 2 salary, -- слагаемые дадут нам или две минимальных или две максимальных, ну или минимальную и максимальную зарплаты, да на 2 поделим, вот и будет или средняя или просто минимум или максимум
                    key_skills
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- уже давно известное условие соответствия вакансии дата-сайентиста
                    AND
                    (salary_from IS NOT NULL OR salary_to IS NOT NULL) -- условие наличия данных хотя бы для минимальной или максимальнной зарплаты
                    ORDER BY skills DESC
             '''

In [76]:
# результат запроса
skills_salary = pd.read_sql_query(query_6_7, connection)
skills_salary

,skills,salary,key_skills
0,NaN,150000,None
1,NaN,160000,None
2,NaN,135000,None
3,NaN,135000,None
4,16.0,290000,SCALA\tJava\tSQL\tАнализ данных\tPython\tElast...
...,...,...,...
57,3.0,350691,Computer Vision\tC/C++\tMachine learning
58,3.0,325000,SQL\tPython\tData Analysis
59,3.0,350691,Computer Vision\tC/C++\tMachine learning
60,3.0,333156,Python/R\tData Scientist\tMachine Learning


8. Предидущий запрос внезапно заставил осознать: как-то подозрительно мало записей с указанными зарплатами, по сравнению с общим количеством вакансий для дата-сайентистов. Как сказал Ленин, после третьего выстрела Каплан: "Чо, опять ?! Два раза было чо-ли мало ?". Давайте выясним сколько вакансий без указания зарплаты. С указанием мы уже знаем - 62 штуки.

In [77]:
# текст запроса
query_6_8 = f'''SELECT 
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- уже давно известное условие соответствия вакансии дата-сайентиста
                    AND salary_from IS NULL AND salary_to IS NULL -- условие отсутствия и минимальной и максимальнной зарплат
             '''

In [78]:
# результат запроса
salary_empty = pd.read_sql_query(query_6_8, connection)
salary_empty

,count
0,418


9. Самые умные и дальновидные, ну или кому денег двать некуда. Работодатели, ждущие дата-сайентистов.

In [79]:
# текст запроса
query_6_9 = f'''SELECT 
                    e.name,
                    COUNT(v.id) vacancies -- подсчет вакансий по их id для каждого работодателя 
                FROM 
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- присоединяем для получения названий 
                WHERE
                    (v.name ~* 'data scientist' OR 
                    v.name ~* 'data science' OR 
                    v.name ~* 'исследователь данных' OR
                    (v.name ~ 'ML' AND v.name !~* 'HTML') OR
                    v.name ~* 'machine learning' OR
                    v.name ~* 'машинн.*обучен.*') -- уже давно известное условие соответствия вакансии дата-сайентиста
                GROUP BY e.id -- группируем по id работодателя
                ORDER BY vacancies DESC -- сортировка по убыванию
             '''

In [80]:
# результат запроса
employers_ds = pd.read_sql_query(query_6_9, connection)
employers_ds

,name,vacancies
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
...,...,...
220,Федеральное автономное учреждение Государствен...,1
221,Представительство ООО «Квалитет» (Российская Ф...,1
222,DIGINETICA,1
223,Дром,1


### Выводы по предметному анализу

* сколько вакансий имеет отношение к данным? 1771 имеет. То есть 3.6 % от всего количества. Грустно становится тем, кто, скован одной цепью и связан одной целью с ней. Да нет, не с царицей полей - кукурузой. А с ней - наукой о данных.
* грустно? а не хотите анафилактический шок от осознания того, сколько есть подходящих вакансий для начинающего дата-сайентиста. А он будет: всего 51 вакансия. Скока - скока? Да, прав был незабвенный Николай Гаврилович: "Что делать?".
* текс, и что же спасет отца русской демократии? Правильно, SQL или postgres. Стоит владеть одним из этих ключевых навыков и уже можно всерьез рассматривать 201 вакансию. Потому, что именно в этом количестве наши дорогие работодатели указали данные ключевые навыки (речь, само собой, о вакансиях для дата-сайентистов).
* а что, если — Не стоит. Ясно. Тогда, может быть, нужно — Не нужно. Понятно. Разрешите хотя бы — А вот это попробуйте. Попробуйте заиметь ключевой навык Python, ведь он весьма популярен - его «желают ощущать» у соискателей работодатели в 351 вакансии. А это, за между прочим, 73 % всех вакансий для дата-сайентистов. Даже если не возьмут – будет, о чем поболтать на собеседовании.
* 6.41. Что это? Без девятнадцати минут семь? Нет. Именно столько ключевых навыков в среднем указывают в вакансиях для дата-сайентистов. Играли бы в "Что? Где? Когда?" против знатоков - приз был бы наш. Никто бы не догадался.
* а вот зайца кому, зайца?! Выбегайца! а вот про зарплату. Вау, прямая корреляция с опытом, чем больше опыт - тем больше денег предлагают. Даст ист фантастиш. "Вот оно чо, Михалыч, вот оно чооо... ". Сотрудникам без опыта готовы платить 75 тыс. рублей (ну как готовы, обещают ...). С опытом от года и до 3 лет - почти в два раза больше, 140 тыс. руб. А от 3 и до 6 лет: вообще в три с половиной раза больше, 243 тыс. руб. Правда стоит отметить, что из 480 вакансий для дата-сайентистов хоть какая-то зарплата указали лишь в 62. А вот в 418 работодатели не соизволили, не смилостивились. Значит строим предположение о корреляции на основании только лишь 13 % вакансий. Маловато будет. Маловато !!! Кстати в целом по базе данных зарплата (минимум, максимум или обе границы) указаны в 49 % случаев (да, да, было уже ранее). В защиту гипотезы стоит отметить, что там корреляция также подтверждается с теми же пропорциями.  И еще - для опыта более 6 лет, зарплату вообще не принято указывать. Но очевидно, что она еще выше, раз в пять - шесть от без опытных?
* "Папа, а есть связь между количеством ключевых навыков и зарплатой? Нет сынок, это фантастика." Нет такой связи. Ну хоть узнали сколько таких навыков указано бывает: от 2 до 16. И подсмотрели максимальную и минимальную указанные зарплаты в вакансиях: 55.5 тыс. руб. и 550 тыс. Ну и распределение, равномерное - равномерное.
* сколько работодателей ждут дата-сайентистов? 225 или 1.5 %. Вот они, которые одной ногой уже в будущем "цифровом мире". Кстати, Яндекс почему-то отсутствует. Наверное, уже насытился ...

# Общие выводы

Мы, бояре — народ работящий, такая уж наша боярская доля. Не поленимся и сделаем общие выводы на основе частных.

Анализ всех вакансий:

* общее количество вакансий 49197, общее количество работодателей 23501, а вот с вакансиями только 14906. Т.о. среднее количество вакансий у работодателя: 3.3 (рассматриваем работодателей с вакансиями). 
* распределение вакансий по регионам, прямо коррелирует с «масштабом региона». В первую сотню по количеству вакансий входят административные центры субъектов Российской Федерации, крупнейшие города помимо них, а также столицы и крупные города стран СНГ. Так на долю Москвы приходится почти 11 % от общего количества вакансий, доля Санкт-Петербурга почти 6 %. На третьем месте Минск с более, чем 4 %. 
* более половины вакансий - 51 %, не имеют каких-либо указаний на предлагаемую заработную плату, не минимума ни максимума. В 21 % имеется полная информация о предложении работодателя по зарплате. В отношении только минимума определились в 23 % вакансий, а максимума в 5 % вакансий. Средние значения для нижней и верхней границы зарплатной вилки составляют 71 и 110 тыс. рублей, соответственно. Минимум нижней и верхней границ предлагаемой зарплаты определить не просто, т.к. очевидно, что в минимальных значениях пропущены нули или разряд "тысячи". Исходя из здравого смысла следует ориентироваться на минимальные значения от 15 - 20 тыс. рублей. Максимум указан для обеих границ один – 1000000 руб. Визуализация показывает, что основная часть вакансий имеет зарплатную вилку в пределах от 10 до 100 тыс. руб.  Еще до 200 тыс. руб. и далее до 300 тыс. руб. наблюдается небольшая резко уменьшающаяся доля вакансий. Далее редкие отдельные «пеньки» от нескольких до единичных вакансий. 
* более 80 % вакансий предполагают личное присутствие на рабочем месте (полный день, сменный и гибкий график), на долю удаленной работы приходится около 19 %. Вахтовый метод примерно 0.8 %. Относительно формата занятости: более 93 % вакансий предполагают полную занятость. Частичная занятость и стажировка - всего чуть больше 4 и полутра процентов, соответственно. Проектная работа, менее 1 %.
* требуемый опыт работы указан в подавляющем количестве вакансий - более 85 %. При этом половина устанавливает опыт от 1 до 3 лет, почти 30 % от 3 до 6 лет и 2.7 % - больше 6 лет опыта. Опыт прямо связан зарплатными предложениями, при росте опыта растет предлагаемая зарплата.
* если посмотреть количество регионов для каждого работодателя, в которых он публикует свои вакансии, то мы видим корреляцию с количеством вакансий у работодателей, т.е. при росте числа вакансий растет число регионов публикации.
* почти 8.5 тыс. работодателей не указали сферы деятельности в которых они предлагают работу, это без малого 36 %.
* связь между количеством ключевых навыков и зарплатой отсутствует.  Ключевых навыков указано от 1 до 30. При этом в 17.2 % вакансий ключевые навыки не указаны.

Анализ вакансий, связанных с «Data Science»:

* 1771 вакансия имеет отношение к данным (в названии присутствуют или «data» или «данн»), то есть 3.6 % от всего количества вакансий. Если говорить о вакансиях для дата-сайентистов (исходя из перечня указанных условий для названий), то таковых 480 или чуть меньше 1 %.
* всего в 51 вакансии указан уровень Junior, это 10.6 % от всех вакансий, для дата-сайентистов.
* SQL или postgres в качестве ключевых навыков указаны в 201 вакансии для дата-сайентистов). Python популяренее – присутствует в 351 вакансии, что составляет 73 % всех вакансий для дата-сайентистов.
* 6.41 среднее количество ключевых навыков в вакансиях для дата-сайентистов. 
* В 13 % (62 вакансии) для дата-сайентистов указана одна или обе границы зарплатной вилки. Сотрудникам без опыта готовы платить 75 тыс. рублей, с опытом от года и до 3 лет почти в два раза больше, 140 тыс. руб., а от 3 и до 6 лет: в три с половиной раза больше, 243 тыс. руб. Для опыта свыше 6 лет – зарплата не указана. Основываясь на информации по всем вакансиям можно считать, что предлагаемая зарплата должна быть в 5-6 раз выше, чем для соискателей без опыта. Максимальная и минимальная указанные зарплаты: 55.5 и 550 тыс. руб.
* связь между количеством ключевых навыков и зарплатой также отсутствует.  Ключевых навыков для дата-сайентистов указано от 2 до 16. Распределение по вакансиям равномерное.

Выводы:

Специальность «Data Science» достаточно редко востребована в общем количестве вакансий ИТ сферы. Вакансии предлагаются скорее крупными компаниями (банки и иные финансовые компании, телекоммуникационные корпорации, интернет-гиганты торговой, социально-сетевой, игровой и т.п. направленности, торговые гиганты и т.п.), крупные промышленные компании и научные учреждения. Востребованы специалисты с опытом, с набором определенных ключевых навыков. Вакансии предлагаются в крупных городах, Москве, СПб. Возможная удаленная работа. Заработная плата в целом в два – три раза выше, чем в среднем для ИТ сферы. И зависит в первую очередь от опыта.

Имеет ли смысл идти в дата-сайентисты. Имеет. Если готов долго упорно учиться и несколько лет нарабатывать опыт с наличием иного источника существования. В конце пройти тяжелый отбор и выдержать высокую конкуренцию.




# Дополнительные исследования

1. Получим информацию о таинственном ИП Поляков Денис Иванович:
* из какого региона этот работодатель,
* в каких сферах деятельности он предлагает вакансии,
* в каких регионах предлагаются вакансии,
* какие именно вакансии (название, график, оплата, регион вакансии и т.д.).

In [81]:
# текст запроса
query_7_1 = f'''SELECT 
                    e.id id_eployer,
                    e.name,
                    a.name area
                FROM 
                    employers e
                    JOIN areas a ON e.area = a.id -- присоединяем только для имени региона
                WHERE 
                    e.id = 5130287 -- мы знаем, что это id ИП Поляков Денис Иванович
             '''

In [82]:
# результат запроса
mr_polyakov = pd.read_sql_query(query_7_1, connection)
mr_polyakov

,id_eployer,name,area
0,5130287,Поляков Денис Иванович,Владивосток


In [83]:
# текст запроса
query_7_2 = f'''SELECT 
                    COUNT(*) indusiries
                FROM 
                    employers_industries e_i
                    LEFT JOIN industries i ON e_i.industry_id = i.id -- присоединяем для вывода названий сфер деятельности (как оказалось зря)
                WHERE 
                    e_i.employer_id = 5130287 -- мы помним, что это id ИП Поляков Денис Иванович
             '''

In [84]:
mr_polyakov_industries = pd.read_sql_query(query_7_2, connection)
mr_polyakov_industries

,indusiries
0,0


In [85]:
# текст запроса
query_7_3 = f'''SELECT 
                    a.name area_vacancy,
                    COUNT(*) quantity
                FROM 
                    vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id -- "просто добавь воды", в смысле просто добавляем сами названия регионов
                WHERE 
                    v.employer_id = 5130287 -- все еще помним, что это Денис Иванович
                GROUP BY a.name -- сгруппируем по названию региона
                ORDER BY quantity DESC, area_vacancy -- отсортируем по количеству вакансий, ни и по названию региона по алфавиту 
             '''

In [86]:
mr_polyakov_areas = pd.read_sql_query(query_7_3, connection)
mr_polyakov_areas

,area_vacancy,quantity
0,Орск,3
1,Балаково,2
2,Бийск,2
3,Братск,2
4,Брянск,2
...,...,...
83,Усолье-Сибирское,1
84,Уссурийск,1
85,Чита,1
86,Южно-Сахалинск,1


In [87]:
# текст запроса
query_7_4 = f'''SELECT 
                    v.name,
                    v.key_skills,
                    v.schedule,
                    v.experience,
                    v.employment,
                    v.salary_from,
                    v.salary_to,
                    a.name area_vacancy
                FROM 
                    vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id -- просто добавляем нормальное название региона вакансии
                WHERE 
                    v.employer_id = 5130287 -- это опять же ИП Поляков ...
                ORDER BY  v.salary_from DESC, v.salary_to DESC -- тут можно по-всякому сортировать и выводить по кусочкам, для удобства анализа
             '''

In [88]:
# результат запроса
mr_polyakov_vacancies = pd.read_sql_query(query_7_4, connection)
mr_polyakov_vacancies

,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_vacancy
0,Компьютерный мастер,Навыки продаж\tНавыки работы с возражениями\tУ...,Полный день,От 1 года до 3 лет,Полная занятость,70000,NaN,Тула
1,Компьютерный мастер,None,Полный день,От 1 года до 3 лет,Полная занятость,60000,NaN,Кемерово
2,Компьютерный мастер,None,Полный день,От 1 года до 3 лет,Полная занятость,60000,NaN,Курск
3,Компьютерный мастер,None,Полный день,От 1 года до 3 лет,Полная занятость,60000,NaN,Оренбург
4,Компьютерный мастер,None,Полный день,От 1 года до 3 лет,Полная занятость,60000,NaN,Набережные Челны
...,...,...,...,...,...,...,...,...
122,Ученик компьютерного мастера,None,Полный день,Нет опыта,Стажировка,30000,50000.0,Геленджик
123,Ученик компьютерного мастера,None,Полный день,Нет опыта,Стажировка,30000,50000.0,Балаково
124,Ученик компьютерного мастера,None,Полный день,Нет опыта,Стажировка,30000,50000.0,Новороссийск
125,Ученик компьютерного мастера,None,Полный день,Нет опыта,Стажировка,30000,50000.0,Тюмень


1. ИП Поляков Денис Иванович зарегистрировано во Владивостоке. Раньше всех просыпается Денис Иванович, если он там, конечно находится. 
2. Каких-либо сфер деятельности для данного работодателя не указано.
3. Вакансии Полякова Д.И. размещены для 88 регионов (почти в каждом регионе России, которых на данный момент 89). Для Орска размещено 3 вакансии (вероятно там острая нехватка компьютерных мастеров - все уехали в Москву), еще в 37 регионах по две вакансии, соответственно, в 50 регионах всего по одной.
4. Всего этот ИП разместил 127 вакансий.
* Вакансии - компьютерный мастер либо ученик компьютерного матера. 
* Предлагаемая зарплата:
   - минимальная для ученика компьютерного мастера 30 тыс. руб., для компьютерного мастера от 40 тыс. руб. в 10 регионах и 50 тыс. в Липецке до 60 тыс. руб. в остальных регионах. В Туле предлагается минимум 70 тыс. рублей, вероятно из-за дороговизны жизни.
   - максимальная в 5 регионах 70 тыс. руб. (Ижевск, Новотроицк, Курск, Нижневартовск, Воткинск) и в 8 регионах 55 тыс. руб. (Смоленск, Ульяновск и т.д.). В остальных - 50 тыс. руб. Особняком стоит Стерлитамак, где можно заработать, наверное, больше, чем в Москве, с максимальной предлагаемой зарплатой в 150 тыс. рублей (при минимуме 40 тыс.).
* Требование по опыту работы, либо отсутствует, либо от 1 до 3 лет, соответственно предлагаемой зарплате и вакансии (ученик или мастер). 
* Для ученика компьютерного мастера предлагается стажировка, для компьютерного мастера - полная занятость. 
* График для всех вакансий указан как полный день.
* Навыки не указаны для всех вакансий, кроме 6 - компьютерный мастер в Архангельске, Тамбове, Орле, Томске, Симферополе и Туле. Требуются навыки: Техники продаж, Уверенный пользователь ПК, Ремонт ПК, Настройка ПО, Настройка ПК, Сборка ПК, Грамотная речь, Выездное обслуживание, Диагностика, Активные продажи, Ремонт электроники, Диагностика ПК, Опытный пользователь ПК. А вот в Стерлитамаке, где можно заработать до 150 тыс. руб., - навыки не нужны.

Так вот ты какой цветочек аленький. В смысле Денис Иванович. 

И вопрос: Who is Mr. Денис Иванович? Какая-то федеральная сеть мастерских по обслуживанию, ремонту компьютерной техники, установке и настройке ПО? Такие объявления на всех подъездах и столбах: мастер на все руки Алеша, недорого, в любой момент, все что угодно …? Или это, тсссссс, прикрытие для резидентуры МИ-6? А может Денис Иванович проводит маркетинговые исследования методом проведения эксперимента для понимания сложных причинно-следственных связей в поведении покупателей, оценки влияния разных факторов, для тестирования гипотез, для глубокого изучения потребителей и для поиска инсайтов? А может Денис Иванович вообще нефилин с планеты Нубиру? «Если наука не имеет достаточных данных для утверждения, что дело номер шестьдесят четыре прибыло к нам, скажем, из ФРГ, то она, наука, на вопрос «Было ли дело за границей?» прямо и недвусмысленно отвечает: вероятно. Если для определения вещественности или существенности дела у науки не хватает фактов, то она, наука, не разводя парадности и шумихи, четко и предельно точно идентифицирует дело как «неизвестное существо, в скобках — возможно, вещество»».

Так кто же ты, Денис Иванович? Вероятно, человек с планеты Земля, вероятно мужчина, вероятно гражданин и уроженец РФ, вероятно индивидуальный предприниматель. Вероятно, прилагающий умственные, организационные, физические, финансовые и иные ресурсы и усилия в сфере оказания разного рода сервисных услуг, связанных с компьютерной техникой и ПО.


2. Попробуем выяснить наличие связи между зарплатой, количеством ключевых требований и требуемым опытом для всей базы вакансий. При этом будем рассматривать вакансии в которых указана хоть какая-то заработная плата (все таки нас интересует связь именно с зарплатой). А для случая отсутствия ключевых навыков приравняем их количество к 0. Ну их же в самом деле 0 (ноль), т.е. нет.

In [89]:
# текст запроса
query_7_5 = f'''SELECT
                    (length(COALESCE(key_skills, '1')) - length(replace(COALESCE(key_skills, '22'), '\t', '')) + 1)::int skills, -- опять вычисляем количество символов разделитетелей, да + 1 (так как разделитетелей на 1 меньше чем разделяемых элементов), только для случая отсутствия ключевых навыков с помощью COALESCE для первого слагаемого будет 1, а для второго 2, а в результате: 1 - 2 + 1 = 0
                    COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from) / 2 salary, -- слагаемые дадут нам или две минимальных или две максимальных, ну или минимальную и максимальную зарплаты, да на 2 поделим, вот и будет или средняя или просто минимум или максимум
                    experience
                FROM
                    vacancies
                WHERE
                    salary_from IS NOT NULL OR salary_to IS NOT NULL -- условие наличия данных хотя бы для минимальной или максимальнной зарплаты
                    ORDER BY experience
             '''

In [90]:
# результат запроса
skills_salary_exp = pd.read_sql_query(query_7_5, connection)
skills_salary_exp

,skills,salary,experience
0,3,600000,Более 6 лет
1,7,250000,Более 6 лет
2,5,147750,Более 6 лет
3,10,750000,Более 6 лет
4,4,300000,Более 6 лет
...,...,...,...
24068,0,90000,От 3 до 6 лет
24069,5,175000,От 3 до 6 лет
24070,13,51869,От 3 до 6 лет
24071,5,83400,От 3 до 6 лет


А теперь сделаем визуализацю и проанализируем результат. Будем использовать тепловую карту с аннотацией, пред этим построив сводную таблицу. Солбцы сводной таблицы - количество ключевых навыков, в стороки - требуемый опыт, т.е. всего 4 варианта строк. В качестве значений возьмем медиану совокупности зарплат для данного количества ключевых навыков и требуемого опыта.

In [91]:
heatmap = pd.pivot_table(
    data = skills_salary_exp,
    values = 'salary',
    index = 'experience',
    columns = 'skills',
    aggfunc='median'
)

heatmap = heatmap.append(heatmap.iloc[0,:]).iloc [1: , :]

heatmap


skills,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
experience,,,,,,,,,,,,,,,,,,,,,
Нет опыта,39000.0,55875.0,52500.0,52500.0,52500.0,57000.0,60000.0,55000.0,52750.0,90000.0,...,45000.0,99000.0,80000.0,NaN,100000.0,46000.0,46000.0,122500.0,NaN,45000.0
От 1 года до 3 лет,80000.0,81000.0,88500.0,90000.0,87672.0,90000.0,90000.0,91332.5,94750.0,95000.0,...,97500.0,150000.0,90000.0,75000.0,91250.0,90000.0,52500.0,62500.0,115000.0,67500.0
От 3 до 6 лет,150000.0,190000.0,261000.0,175000.0,172500.0,150000.0,175000.0,180000.0,175172.5,165000.0,...,300000.0,165000.0,128149.0,150000.0,225000.0,200000.0,287500.0,172500.0,263019.0,112000.0
Более 6 лет,165000.0,217500.0,169500.0,255000.0,225000.0,183070.0,305000.0,289320.0,162500.0,170000.0,...,305000.0,186909.0,380754.5,225000.0,460000.0,NaN,NaN,90000.0,NaN,77140.0


In [92]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


# тепловая карта по сводной таблице
fig = go.Figure(data=go.Heatmap(
                   x=heatmap.columns,
                   y=heatmap.index,
                   z=heatmap,
                   hoverongaps = True,
                   name=''))

# параметры отображения при наведении курсора
fig.update_traces(hoverinfo="all", 
                  hovertemplate="Количество навыков: %{x}<br>Опыт: %{y}<br>Зарплата: %{z}")

# отступы, параметры заголовка и легенды
fig.update_layout(margin=dict(l=20, r=20, t=90, b=20), 
                  title='Взаимосвязь между требуемым опытом, количеством ключевых навыков и зарплатой', 
                  title_x = 0.5, title_font_size=25,
                  legend_orientation="h", legend=dict(x=.5, xanchor="center"))

# параметры названий осей, размеров и шрифтов
fig.update_xaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='Количество ключевых навыков',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='Требуемый опыт',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

<center> <img src = images\plot_7_5.png alt="drawing" style="width:2000px;">

Вывод: количество ключевых навыков не влияет на уровень заработной платы. Влияет требуемый опыт, причем однозначно, вне зависимости от навыков. Есть 5 "черных дыр" - нет такого сочетания количества ключевых навыков и опыта - ничего страшного, на понимание ситуации не влияет.

In [93]:
# Вот и все, что останется после меня ...
connection.close()